In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf
import numpy as np

# Import our input dataset
df = pd.read_excel('Percent_Total_Homeless_DroppedColumns_AllCoC.xlsx')
df.head()

,Year,TotalHomeless,Population,Unemployment,Unsheltered_perc_tot,TotalBeds_perc_tot,Emergency_perc_tot,Transitional_perc_tot,SafeHaven_perc_tot,PermanentSupportive_perc_tot,PermanentOther_perc_tot,RapidRehousing_perc_tot
0,2019,1111,288000,4.4,8.730873,92.979298,71.917192,21.062106,0.0,54.095410,6.390639,15.391539
1,2018,1094,290521,5.0,8.592322,93.967093,68.464351,25.502742,0.0,55.210238,6.489945,11.425960
2,2017,1128,294338,5.4,13.741135,90.691489,52.925532,37.765957,0.0,50.620567,6.294326,3.812057
3,2016,1105,297249,5.3,21.719457,91.131222,50.045249,41.085973,0.0,39.276018,8.778281,4.977376
4,2015,1208,297547,4.9,14.817881,84.354305,49.917219,34.437086,0.0,40.728477,5.877483,2.317881


In [2]:
# df2 = df.drop(['Year', 'TotalBeds', 'TotalHomeless'], axis=1)
# df2.head()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199 entries, 0 to 2198
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Year                          2199 non-null   int64  
 1   TotalHomeless                 2199 non-null   int64  
 2   Population                    2199 non-null   int64  
 3   Unemployment                  2199 non-null   float64
 4   Unsheltered_perc_tot          2199 non-null   float64
 5   TotalBeds_perc_tot            2199 non-null   float64
 6   Emergency_perc_tot            2199 non-null   float64
 7   Transitional_perc_tot         2199 non-null   float64
 8   SafeHaven_perc_tot            2199 non-null   float64
 9   PermanentSupportive_perc_tot  2199 non-null   float64
 10  PermanentOther_perc_tot       2199 non-null   float64
 11  RapidRehousing_perc_tot       2199 non-null   float64
dtypes: float64(9), int64(3)
memory usage: 206.3 KB


In [4]:
# q = df['Unsheltered_perc_tot'].quantile(np.arange(10) / 10)

In [5]:
# df['UnshelteredPercentQuantile'] = df['Unsheltered_perc_tot'].apply(lambda x : q.index[np.searchsorted(q, x, side='right')-1])

In [6]:
pd.cut(df['Unsheltered_perc_tot'], bins=3).value_counts()

(-0.0961, 32.033]    1500
(32.033, 64.065]      484
(64.065, 96.098]      215
Name: Unsheltered_perc_tot, dtype: int64

In [7]:
size_bins=[-1, 33, 66, 100]
group_names = ["0", "1", "2"]

In [8]:
df['Unsheltered_thirds'] = pd.cut(df['Unsheltered_perc_tot'], size_bins, labels=group_names)

In [9]:
df['Unsheltered_thirds'] = df['Unsheltered_thirds'].astype(int)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199 entries, 0 to 2198
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Year                          2199 non-null   int64  
 1   TotalHomeless                 2199 non-null   int64  
 2   Population                    2199 non-null   int64  
 3   Unemployment                  2199 non-null   float64
 4   Unsheltered_perc_tot          2199 non-null   float64
 5   TotalBeds_perc_tot            2199 non-null   float64
 6   Emergency_perc_tot            2199 non-null   float64
 7   Transitional_perc_tot         2199 non-null   float64
 8   SafeHaven_perc_tot            2199 non-null   float64
 9   PermanentSupportive_perc_tot  2199 non-null   float64
 10  PermanentOther_perc_tot       2199 non-null   float64
 11  RapidRehousing_perc_tot       2199 non-null   float64
 12  Unsheltered_thirds            2199 non-null   int32  
dtypes: 

In [11]:
df=df.drop(['Unsheltered_perc_tot', 'Year'], axis=1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2199 entries, 0 to 2198
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   TotalHomeless                 2199 non-null   int64  
 1   Population                    2199 non-null   int64  
 2   Unemployment                  2199 non-null   float64
 3   TotalBeds_perc_tot            2199 non-null   float64
 4   Emergency_perc_tot            2199 non-null   float64
 5   Transitional_perc_tot         2199 non-null   float64
 6   SafeHaven_perc_tot            2199 non-null   float64
 7   PermanentSupportive_perc_tot  2199 non-null   float64
 8   PermanentOther_perc_tot       2199 non-null   float64
 9   RapidRehousing_perc_tot       2199 non-null   float64
 10  Unsheltered_thirds            2199 non-null   int32  
dtypes: float64(8), int32(1), int64(2)
memory usage: 180.5 KB


In [13]:
# Remove outcome target from features data
y = df.Unsheltered_thirds.values
X = df.drop(columns="Unsheltered_thirds").values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [14]:
# Preprocess numerical data for neural network

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=64, activation="sigmoid", input_dim=10))
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
52/52 [==============================] - 0s 896us/step - loss: 0.5784 - accuracy: 0.7083
Epoch 2/100
52/52 [==============================] - 0s 1ms/step - loss: 0.3692 - accuracy: 0.7592
Epoch 3/100
52/52 [==============================] - 0s 1ms/step - loss: 0.1923 - accuracy: 0.7629
Epoch 4/100
52/52 [==============================] - 0s 1ms/step - loss: 0.0649 - accuracy: 0.7592
Epoch 5/100
52/52 [==============================] - 0s 1ms/step - loss: -0.0323 - accuracy: 0.7526
Epoch 6/100
52/52 [==============================] - 0s 1ms/step - loss: -0.1072 - accuracy: 0.7489
Epoch 7/100
52/52 [==============================] - 0s 1ms/step - loss: -0.1749 - accuracy: 0.7477
Epoch 8/100
52/52 [==============================] - 0s 1ms/step - loss: -0.2423 - accuracy: 0.7453
Epoch 9/100
52/52 [==============================] - 0s 1ms/step - loss: -0.3247 - accuracy: 0.7471
Epoch 10/100
52/52 [==============================] - 0s 1ms/step - loss: -0.3912 - accuracy: 0.7423
E

52/52 [==============================] - 0s 1ms/step - loss: -128.8280 - accuracy: 0.7520
Epoch 82/100
52/52 [==============================] - 0s 1ms/step - loss: -133.4387 - accuracy: 0.7550
Epoch 83/100
52/52 [==============================] - 0s 1ms/step - loss: -137.9543 - accuracy: 0.7556
Epoch 84/100
52/52 [==============================] - 0s 1ms/step - loss: -142.5983 - accuracy: 0.7586
Epoch 85/100
52/52 [==============================] - 0s 1ms/step - loss: -147.4045 - accuracy: 0.7550
Epoch 86/100
52/52 [==============================] - 0s 1ms/step - loss: -152.1294 - accuracy: 0.7550
Epoch 87/100
52/52 [==============================] - 0s 1ms/step - loss: -156.9283 - accuracy: 0.7544
Epoch 88/100
52/52 [==============================] - 0s 1ms/step - loss: -161.6274 - accuracy: 0.7532
Epoch 89/100
52/52 [==============================] - 0s 1ms/step - loss: -166.5258 - accuracy: 0.7514
Epoch 90/100
52/52 [==============================] - 0s 1ms/step - loss: -171.5127 - 